In [ ]:
import sys
sys.path.insert(0, '..')

In [ ]:
from XGNN.DiscriminativeApproach.DiscriminativeExplainer import DiscriminativeExplainer

In [ ]:
label_to_explain = 2

In [ ]:
from XGNN.Infrastructure.graph_neural_network import HGNN
import torch

In [ ]:
from XGNN.Infrastructure.Evaluator import Evaluator
evaluator = Evaluator(dataset)

In [ ]:
hgnn = HGNN(d=dataset)
for _ in range(20):
    print(_, hgnn.train_model())

In [ ]:
pred = hgnn.predict_all(new_data=dataset)

gnn_results = set()
for i in pred.keys():
    for ii in range(len(pred[i])):
        if pred[i][ii] == label_to_explain:
            gnn_results.add((ii, i))

In [ ]:
from XGNN.Infrastructure.BaseExplainer import BaseExplainer
base_explainer = BaseExplainer(gnn=hgnn, data=dataset)
base_result = base_explainer.explain(label_to_explain)
base_result

In [ ]:
from ontolearn.owlapy.render import DLSyntaxObjectRenderer
renderer = DLSyntaxObjectRenderer()
print(renderer.render(base_result))

In [ ]:
explainer = DiscriminativeExplainer(hgnn, dataset, "http://example.org/", owl_graph_path = "./owlGraphs/example4.owl", generate_new_owl_file=True)

In [ ]:
from ontolearn.metrics import Accuracy, F1
hypotheses, model = explainer.explain(2, 5, debug=True, max_runtime=60, num_generations=400, quality_func=F1())

In [ ]:
[print(h) for h in hypotheses]

In [ ]:
discriminative_result = hypotheses[0].concept
print(renderer.render(discriminative_result))

In [ ]:
from rich.table import Table
table = Table("Approach", "Precision", "Recall", "Accuracy", "F1 Score", "Explanation", show_lines=True)

In [ ]:
base_score = evaluator.explanation_accuracy(gnn_results, base_result)
base_f1 = evaluator.f1_score(gnn_results, base_result)
table.add_row("Base Explanation",
                f'{base_score[0] * 100:.2f}%',
                f'{base_score[1] * 100:.2f}%',
                f'{base_score[2] * 100:.2f}%',
                f'{base_f1:.4f}',
                renderer.render(base_result))

In [ ]:
for discriminative_result in hypotheses:
    discriminative_score = evaluator.explanation_accuracy(gnn_results, discriminative_result.concept)
    discriminative_f1 = evaluator.f1_score(gnn_results, discriminative_result.concept)
    table.add_row("Discriminative Explanation",
                    f'{discriminative_score[0] * 100:.2f}%',
                    f'{discriminative_score[1] * 100:.2f}%',
                    f'{discriminative_score[2] * 100:.2f}%',
                    f'{discriminative_f1:.4f}',
                    renderer.render(discriminative_result.concept))

In [ ]:
from rich.console import Console
console = Console()
console.print(table)

In [ ]:
from collections import defaultdict
import re, torch

# Read the file and tokenize each line into words
file_path = "D:\XGNN\HAN-master\data\DBLP_four_area\paper.txt"  # Replace with the path to your file
bag_of_words = []

# Vocabulary to store unique words
vocabulary = set()

with open(file_path, "r") as file:
    for idx, line in enumerate(file):
        line = line.strip()
        id, text = line.split("\t", 1)
        words = re.findall(r'\w+', text.lower())  # Tokenize and convert to lowercase
        word_count = defaultdict(int)
        for word in words:
            vocabulary.add(word)  # Add word to vocabulary
            word_count[word] += 1
        bag_of_words.append({"id": id, "word_count": dict(word_count)})

# Create a tensor for each document
tensors = []

for item in bag_of_words:
    vector = [item["word_count"].get(word, 0) for word in vocabulary]
    tensor = torch.tensor(vector, dtype=torch.float32)
    tensors.append({"id": item["id"], "tensor": tensor})



In [ ]:
# Optionally, print some tensors to check outputs
for tensor in tensors[:5]:  # Print only the first 5 to avoid too much output
    print(tensor)

In [ ]:
tensors[0]['tensor'].size()

In [ ]:
import csv
from collections import defaultdict, Counter
import re
import torch

file_path = "D:\\XGNN\\HAN-master\\data\\DBLP_four_area\\paper.txt"  # Ensure path uses double backslashes or raw string
output_csv_path = "output.csv"

# Function to process each line and update global vocabulary
def process_line(line, id_to_counts, vocab, vocab_limit=10000):
    id, text = line.split("\t", 1)
    words = re.findall(r'\w+', text.lower())
    word_count = Counter(words)

    # Update vocabulary only if below limit
    if len(vocab) < vocab_limit:
        vocab.update(word_count.keys())

    # Accumulate word counts for each ID
    id_to_counts[id].update(word_count)

# Main processing block
vocabulary = {}
id_to_counts = defaultdict(Counter)

with open(file_path, "r") as file:
    for line in file:
        process_line(line, id_to_counts, vocabulary)

# Write accumulated counts to CSV file
with open(output_csv_path, "w", newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(vocabulary.keys())  # Write header row with words as column names
    for id, counts in id_to_counts.items():
        row = [counts[word] for word in vocabulary]
        writer.writerow(row)


In [ ]:
bag_of_words_tensor

In [ ]:
with open('output_file.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write feature names as the header
    writer.writerow([''] + feature_names)
    # Write sorted numbers and bag-of-words tensor rows
    for number, row in zip(sorted_numbers, bag_of_words_tensor):
        writer.writerow([number] + row.tolist())

print("Output written to 'output_file.csv'")

In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from nltk.corpus import stopwords
 
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ajayu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from nltk.corpus import stopwords

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ajayu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True